<a href="https://colab.research.google.com/github/jmaly1831/Node-Project-Final/blob/main/Copy_of_Maly_Sprott_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A Look at Gas Prices During the COVID Pandemic
Made by Erica Sprott and Jacquelyn Maly, 2021


In [ ]:
import pandas as pd
import numpy as np
import plotly
import plotly.express as px

# 1. Preparing the Data

In [ ]:
#source for EPA data, using CO daily levels, but the file size is too big for github upload (34 mb): https://aqs.epa.gov/aqsweb/airdata/download_files.html#Daily
#https://www.kaggle.com/mruanova/us-gasoline-and-diesel-retail-prices-19952021 is the source for gas prices, has info on each variable
gas = pd.read_csv('https://github.com/sprotterica/nodeproject/blob/main/PET_PRI_GND_DCUS_NUS_W.csv?raw=True')


This data, gathered from kaggle, includes gas prices from 1995 through 2021. Though it includes a column with average gas price for all formulations and grades, it can also break down the data into those subcategories.
The included grades are regular, midrange, and premium; while the included formulations are conventional and reformulated.
Data is specific to the United States.

In [ ]:
gas.head()

,Date,A1,A2,A3,R1,R2,R3,M1,M2,M3,P1,P2,P3,D1
0,01/02/1995,1.127,1.104,1.231,1.079,1.063,1.167,1.170,1.159,1.298,1.272,1.250,1.386,1.104
1,01/09/1995,1.134,1.111,1.232,1.086,1.070,1.169,1.177,1.164,1.300,1.279,1.256,1.387,1.102
2,01/16/1995,1.126,1.102,1.231,1.078,1.062,1.169,1.168,1.155,1.299,1.271,1.249,1.385,1.100
3,01/23/1995,1.132,1.110,1.226,1.083,1.068,1.165,1.177,1.165,1.296,1.277,1.256,1.378,1.095
4,01/30/1995,1.131,1.109,1.221,1.083,1.068,1.162,1.176,1.163,1.291,1.275,1.255,1.370,1.090


First we want to rename the columns to make them easier to understand. In the following table,

*   g = grade
*   f = formulation
*   c = conventional
*   r = reformulated
*   reg = regular
*   mid = midgrade
*   pre = premium




In [ ]:
#rename columns
gas = gas.rename(columns={'A1':'allg_allf', 'A2':'allg_c', 'A3': 'allg_r', 'R1': 'reg_allf', 'R2':'reg_c', 'R3': 'reg_r', 'M1':'mid_allf',
                          'M2': 'mid_c', 'M3':'mid_r', 'P1':'pre_allf', 'P2': 'pre_c', 'P3': 'pre_r', 'D1': 'diesel'})

In [ ]:
gas.head()

,Date,allg_allf,allg_c,allg_r,reg_allf,reg_c,reg_r,mid_allf,mid_c,mid_r,pre_allf,pre_c,pre_r,diesel
0,01/02/1995,1.127,1.104,1.231,1.079,1.063,1.167,1.170,1.159,1.298,1.272,1.250,1.386,1.104
1,01/09/1995,1.134,1.111,1.232,1.086,1.070,1.169,1.177,1.164,1.300,1.279,1.256,1.387,1.102
2,01/16/1995,1.126,1.102,1.231,1.078,1.062,1.169,1.168,1.155,1.299,1.271,1.249,1.385,1.100
3,01/23/1995,1.132,1.110,1.226,1.083,1.068,1.165,1.177,1.165,1.296,1.277,1.256,1.378,1.095
4,01/30/1995,1.131,1.109,1.221,1.083,1.068,1.162,1.176,1.163,1.291,1.275,1.255,1.370,1.090


Next, we want to change the date column to a datetime object in pandas.

In [ ]:
#change Date to pandas datetime 
gas['Date'] = pd.to_datetime(gas['Date'])

Finally, we want to check and see if there are any missing variables. According to the output below, there are not any missing variables, so the data cleaning process for this dataset is pretty easy.

In [ ]:
#check to see if there are missing variables
gas.isna().sum()

Date         0
allg_allf    0
allg_c       0
allg_r       0
reg_allf     0
reg_c        0
reg_r        0
mid_allf     0
mid_c        0
mid_r        0
pre_allf     0
pre_c        0
pre_r        0
diesel       0
dtype: int64

Finally, let's take a look at the shape of the gas dataset. According to the output below, there are 14 variables and 1361 entries. We can look at a sample of the gas dataset to see a random entry.

In [ ]:
print(gas.shape)
print(gas.columns.values)
gas.sample()

(1361, 14)
['Date' 'allg_allf' 'allg_c' 'allg_r' 'reg_allf' 'reg_c' 'reg_r'
 'mid_allf' 'mid_c' 'mid_r' 'pre_allf' 'pre_c' 'pre_r' 'diesel']


,Date,allg_allf,allg_c,allg_r,reg_allf,reg_c,reg_r,mid_allf,mid_c,mid_r,pre_allf,pre_c,pre_r,diesel
1068,2015-06-22,2.895,2.803,3.082,2.812,2.72,3.004,3.006,2.902,3.206,3.182,3.109,3.317,2.859


# 2. Taking a sample for data visualization
We chose to take a sample of teh last 1500 observations because it was more relevant for our timeline -- we really wanted to focus on the effect of the COVID pandemic with some context of gas price trends from the preceeding years.




In [ ]:
gas_sample = gas.tail(1500)
gas_sample.head()

,Date,allg_allf,allg_c,allg_r,reg_allf,reg_c,reg_r,mid_allf,mid_c,mid_r,pre_allf,pre_c,pre_r,diesel
0,1995-01-02,1.127,1.104,1.231,1.079,1.063,1.167,1.170,1.159,1.298,1.272,1.250,1.386,1.104
1,1995-01-09,1.134,1.111,1.232,1.086,1.070,1.169,1.177,1.164,1.300,1.279,1.256,1.387,1.102
2,1995-01-16,1.126,1.102,1.231,1.078,1.062,1.169,1.168,1.155,1.299,1.271,1.249,1.385,1.100
3,1995-01-23,1.132,1.110,1.226,1.083,1.068,1.165,1.177,1.165,1.296,1.277,1.256,1.378,1.095
4,1995-01-30,1.131,1.109,1.221,1.083,1.068,1.162,1.176,1.163,1.291,1.275,1.255,1.370,1.090


# 3. Data Visualization

Let's first take a look at all gas prices for all formulas and grades between 1995 and 2021. This will give us some context for what the trends in gas prices have been over the last 3 decades. Then we can split up the data into different formulations and different grads to see if any trends exist there. 

In [ ]:
plot = px.scatter(gas_sample, x='Date', y='allg_allf',
                  opacity=0.3,  
                 marginal_y = 'histogram', 
                 hover_name='Date', hover_data=['allg_allf'])
plot.show()

Let's take a smaller sample so we can focus on the last few years.

In [ ]:

sample2 = gas.tail(200)
plot2 = px.scatter(sample2, x='Date', y='allg_allf', 
                 opacity=0.3,  
                 marginal_y = 'histogram', 
                 hover_name='Date', hover_data=['allg_allf'],
                )
plot2.show()

Now that we're focusing on the last four years, we can see a huge dip at the beginning of 2020 as quarantine begins. Overall gas prices drop below 2 dollars per gallon in early April of 2020. Our hover data shows us that this tipping point came between April 6th and April 13th, where prices dropped from 2.022 to 1.95 dollars per gallon.

What if we split up the trends into different formulations and grades? First, we need to melt the dataframe so we can show each different value on the y axis at the same time. We'll keep Date as a column as the identifier variable.





In [ ]:
melted = pd.melt(sample2, id_vars='Date')
melted.head()

,Date,variable,value
0,2017-04-03,allg_allf,2.471
1,2017-04-10,allg_allf,2.534
2,2017-04-17,allg_allf,2.546
3,2017-04-24,allg_allf,2.559
4,2017-05-01,allg_allf,2.522


In [ ]:
melt_all = pd.melt(gas, id_vars = 'Date')

Then, plot by date with each different gas variation in beautiful rainbow colors.

In [ ]:
plot3 = px.scatter(melted, x= 'Date', y= 'value',
                 opacity=0.3,   
                 color = 'variable',
                 color_discrete_sequence=plotly.colors.qualitative.Prism,
                 title='Gas Prices in the US Over Time',
                 labels={'value':'Price per Gallon ($)'},
                )
plot3.show()

The Recession variable has been added to the graphs in reference to our machine learning model below.

In [ ]:
plot5 = px.line(melted, x= 'Date', y= 'value',   
                 color = 'variable',
                color_discrete_sequence=plotly.colors.qualitative.Prism,
                title='Gas Prices in the US Over Time',
                 labels={'value':'Price per Gallon ($)'},
                )
plot5.show()

In [ ]:
plot6 = px.line(melt_all, x= 'Date', y= 'value',   
                 color = 'variable',
                color_discrete_sequence=plotly.colors.qualitative.Prism,
                title='Gas Prices in the US Over Time',
                 labels={'value':'Price per Gallon ($)'},
                )
plot6.show()

Gas prices were already falling in winter of 2019-2020 because it was one of the warmest winters on record -- people didn't need to use as much gas for things like heating their homes. Of course, this trend continued into spring of 2020 as people stayed home to quarantine.

Finally, we can see that almost all the gas prices follow the same trend except one. We found that deisel prices didn't drop as much during the COVID lockdown as the other gas prices did. Let's isolate deisel trends specifically in another graph below.

In [ ]:
plot4 = px.scatter(sample2, x='Date', y='diesel', 
                 opacity=0.3,  
                 marginal_y = 'histogram', 
                 hover_name='Date', hover_data=['diesel'],
                 title='Diesel Prices in the US Over Time',
                 labels={'value':'Price per Gallon ($)'},
                )
plot4.show()

As we can see above, diesel prices didn't experience the same severe dip like other gas prices did. Overall, it didn't drop much below 2.4 on average. This could be due to a less elastic demand for diesel. Large and long distance trucks often use diesel because it provides more torque. If trucking was considered an essential business during lockdown, this could explain this smaller impact on gas prices.

# 4. Recession Data
Using information from the NBER's Business Cycle Dating Committee, we can create a dummy variable that indicates the following: 1 when in a recession, 0 when not in a recession. 

Our goal is, using this variable in addition to the gas prices data, to create a machine learning algorithm to predict the recessionary status of the US economy. 

In [ ]:
from datetime import datetime

In [ ]:
dates = ['2001-03-05','2001-03-12','2001-03-19','2001-03-26', 
         '2001-04-02','2001-04-09','2001-04-16','2001-04-23','2001-04-30', 
         '2001-05-07','2001-05-14','2001-05-21','2001-05-28',
         '2001-06-04','2001-06-11','2001-06-18','2001-06-25', 
         '2001-07-02','2001-07-09','2001-07-16','2001-07-23','2001-07-30', 
         '2001-08-06','2001-08-13','2001-08-20','2001-08-27', 
         '2001-09-03','2001-09-10','2001-09-17','2001-09-24', 
         '2001-10-01','2001-10-08','2001-10-15','2001-10-22','2001-10-29', 
         '2001-11-05','2001-11-12','2001-11-19','2001-11-26', 
         '2007-12-03','2007-12-10','2007-12-17','2007-12-24', '2007-12-31',
         '2008-01-07','2008-01-14','2008-01-21','2008-01-28',
         '2008-02-04','2008-02-11','2008-02-18','2008-02-25',
         '2008-03-03','2008-03-10','2008-03-17','2008-03-24','2008-03-31',
         '2008-04-07', '2008-04-14','2008-04-21','2008-04-28',
         '2008-05-05','2008-05-12','2008-05-19','2008-05-26',
         '2008-06-02','2008-06-09','2008-06-16','2008-06-23','2008-06-30',
         '2008-07-07','2008-07-14','2008-07-21','2008-07-28',
         '2008-08-04','2008-08-11','2008-08-18','2008-08-25',
         '2008-09-01','2008-09-08','2008-09-15','2008-09-22','2008-09-29',
         '2008-10-06','2008-10-13','2008-10-20','2008-10-27',
         '2008-11-03', '2008-11-10','2008-11-17','2008-11-24',
         '2008-12-01','2008-12-08','2008-12-15','2008-12-22','2008-12-29',
         '2009-01-05','2009-01-12','2009-01-19','2009-01-26',
         '2009-02-02','2009-02-09','2009-02-16','2009-02-23',
         '2009-03-02','2009-03-09','2009-03-16','2009-03-23','2009-03-30',
         '2009-04-06', '2009-04-13','2009-04-20','2009-04-27',
         '2020-02-03', '2020-02-10','2020-02-17','2020-02-24',
         '2020-03-02','2020-03-09','2020-03-16','2020-03-23','2020-03-30',
         '2020-04-06','2020-04-13','2020-04-20','2020-04-27',
         '2020-05-04','2020-05-11','2020-05-18','2020-05-25',
         '2020-06-01','2020-06-08','2020-06-15','2020-06-22','2020-06-29',
         '2020-07-06','2020-07-13','2020-07-20','2020-07-27',
         '2020-08-03','2020-08-10','2020-08-17','2020-08-24','2020-08-31',
         '2020-09-07','2020-09-14','2020-09-21','2020-09-28',
         '2020-10-05','2020-10-12','2020-10-19','2020-10-26',
         '2020-11-02','2020-11-09','2020-11-16','2020-11-23','2020-11-30',
         '2020-12-07','2020-12-14','2020-12-21','2020-12-28',
         '2021-01-04','2021-01-11','2021-01-18','2021-01-25']

dates_list = [datetime.strptime(date, '%Y-%m-%d').date() for date in dates]
gas['Recession'] = gas['Date'].isin(dates_list).astype(int)




In [ ]:
gas.head()

,Date,allg_allf,allg_c,allg_r,reg_allf,reg_c,reg_r,mid_allf,mid_c,mid_r,pre_allf,pre_c,pre_r,diesel,Recession
0,1995-01-02,1.127,1.104,1.231,1.079,1.063,1.167,1.170,1.159,1.298,1.272,1.250,1.386,1.104,0
1,1995-01-09,1.134,1.111,1.232,1.086,1.070,1.169,1.177,1.164,1.300,1.279,1.256,1.387,1.102,0
2,1995-01-16,1.126,1.102,1.231,1.078,1.062,1.169,1.168,1.155,1.299,1.271,1.249,1.385,1.100,0
3,1995-01-23,1.132,1.110,1.226,1.083,1.068,1.165,1.177,1.165,1.296,1.277,1.256,1.378,1.095,0
4,1995-01-30,1.131,1.109,1.221,1.083,1.068,1.162,1.176,1.163,1.291,1.275,1.255,1.370,1.090,0


In [ ]:
gas.tail()

,Date,allg_allf,allg_c,allg_r,reg_allf,reg_c,reg_r,mid_allf,mid_c,mid_r,pre_allf,pre_c,pre_r,diesel,Recession
1356,2020-12-28,2.330,2.225,2.535,2.243,2.158,2.423,2.634,2.482,2.858,2.889,2.770,3.031,2.635,1
1357,2021-01-04,2.336,2.227,2.549,2.249,2.160,2.437,2.639,2.484,2.867,2.895,2.771,3.042,2.640,1
1358,2021-01-11,2.403,2.298,2.610,2.317,2.232,2.498,2.702,2.550,2.927,2.959,2.839,3.101,2.670,1
1359,2021-01-18,2.464,2.351,2.688,2.379,2.285,2.579,2.759,2.601,2.995,3.014,2.885,3.166,2.696,1
1360,2021-01-25,2.478,2.363,2.703,2.392,2.298,2.593,2.776,2.615,3.014,3.033,2.900,3.191,2.716,1


Next, import all the necessarry packages to use Decision Tree Classifier, and treat Recession as our target variable.


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier()

datelessgas = gas.drop(columns=['Date'])

In [ ]:
X = datelessgas.drop(columns=['Recession'])
y = datelessgas['Recession']

Training and testing our model: we decided to train our model on 33% of the gas price data. 

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:


clf.fit(X_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [ ]:

predicted = clf.predict(X_test)
actual = np.array(y_test)

print('Look at first 10 predictions:')
print('Predicted: ',predicted[:10])
print('Actual:    ',actual[:10])

Look at first 10 predictions:
Predicted:  [0 0 1 0 1 0 0 0 0 0]
Actual:     [0 0 1 0 0 0 0 0 0 0]


In [ ]:
accuracy_score(predicted,actual)

0.9311111111111111

Using gas prices AND dates, the Decision Tree Classifier was able to predict the presence of recessions with an accuracy of 99%

Using only gas prices, the Decision Tree Classifier was able to predict the presence of recessions with an accuracy of 93%
